In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

import pyopia.classify
import pyopia.process
import pyopia.statistics
import pyopia.pipeline
import pyopia.instrument.silcam
import pyopia.io
import pyopia.tests.testdata

In [2]:
model_path = pyopia.tests.testdata.get_example_model(os.getcwd())

In [3]:
toml_settings = pyopia.io.load_toml('config.toml')

In [4]:
classifier = pyopia.classify.Classify(model_path=toml_settings['steps']['classifier']['model_path'])

In [5]:
filename = pyopia.tests.testdata.get_example_silc_image(os.getcwd())

image = np.load(filename, allow_pickle=False)

imc = np.float64(image)

# simplify processing by squeezing the image dimensions into a 2D array
# min is used for squeezing to represent the highest attenuation of all wavelengths
imc = np.min(imc, axis=2)
imc /= 255

pyopia.plotting.show_imc(imc, 24)

AttributeError: module 'pyopia' has no attribute 'plotting'

In [ ]:
# Initialise the pipeline and run the initial steps
processing_pipeline = pyopia.pipeline.Pipeline(toml_settings)

# Load an image (from the test suite)
filename = pyopia.tests.testdata.get_example_silc_image(os.getcwd())

# Process the image to obtain the stats dataframe
stats = processing_pipeline.run(filename)

In [ ]:
import pyopia.plotting


filename
im = pyopia.instrument.silcam.load_image(filename)
pyopia.plotting.show_imc(im, 24)

Note: the returned stats from `stats = processing_pipeline.run(filename)` are single-image only and not appended if you loop through several filenames! It is recommended to use this step as part of pipeline that uses {class}`pyopia.io.StatsToDisc` for properly appending data into NetCDF format when processing several files.

## The STATS DataFrame

This is the main [Pandas DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html#pandas-dataframe) containing the processed information about every particle measured.

This does not contain any calibrated values, so dimentions (e.g. equivalent_diameter etc.) and positions of ROI bounding boxes (e.g. minr etc.) are all in pixels (not microns). This allows for altering pixels size without having to re-process if a post-calibration is performed on the data, for example.

The position of each particle within the original raw image are given by the bounding box at location (minr, minc, maxr, maxc) - with r and c being rows and columns, respectively.

Classification probabilities are given by columns with 'probability_*'. Note: If `[steps.classifier]`is not defined in the config, the classification will be skipped and no probabilities reported. To use PyOPIA's Classification module requires the extra dependencies (pip install pyopia[classification] or pip install pyopia[classification-arm64])

In [ ]:
# print the stats DataFrame
stats.head()

In [ ]:
# Calculate the volume distribution from the stats DataFrame.
# (Usually several images would be needed for statistics to converge.
# This can be done by appending new image stats to the DataFrame)
dias, vd = pyopia.statistics.vd_from_stats(stats, 24)

# plot the volume distribution
plt.plot(dias, vd)
plt.xscale('log')
plt.xlabel('ECD [um]')
plt.ylabel('Volume Concentration [uL/sample vol.]')
plt.title('Particle Volume Distribution')
plt.show()